Importações

In [2]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym
from math import *


TODO: 3o item
      comparacoes/analises
      talvez mudar tudo para não usar sympy

## ITEM 1

In [102]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
f1 = x1*x2 +x1 -3*x5
f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    
F = np.transpose(np.array([f1,f2,f3,f4,f5]))

In [103]:
## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
    return np.array(Fx)

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3), f.diff(x4), f.diff(x5)])
        
    return np.array(Jmatrix)    


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
        i += 1
        
        
    return np.array(Jx)

## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.



### Método de Newton

In [104]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    print("Começando com Xo = ", x)
    while k <= maxIterations:
        Fx = [-x for x in calcF(F,x)]
        Jx = calcJ(J(F), x)
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(Fx, np.float64))
        x = x + y
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            k += 1
        
    return output

In [105]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodNewton(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")


Começando com Xo =  [10 10 10 10 10]
1  Iteração x= [0.01065204 9.99922036 5.00102591 5.03641579 0.03645869] Norma:  15.769785630785213
2  Iteração x= [ 0.01001153 10.01012371  2.49920776  2.59112275  0.03674507] Norma:  3.498381268945265
3  Iteração x= [9.98373148e-03 1.00839478e+01 1.24338306e+00 1.43787284e+00
 3.68870704e-02] Norma:  1.7066139146402113
4  Iteração x= [9.68920699e-03 1.04317749e+01 6.06359646e-01 9.75892034e-01
 3.69557589e-02] Norma:  0.8603539169025427
5  Iteração x= [8.50591251e-03 1.18477258e+01 2.74174277e-01 8.66848855e-01
 3.69857062e-02] Norma:  1.4584772627650562
6  Iteração x= [5.41646124e-03 1.67133282e+01 1.03992448e-01 8.60022364e-01
 3.69918658e-02] Norma:  4.868583458334746
7  Iteração x= [2.92103470e-03 2.76408171e+01 6.02819450e-02 8.59724155e-01
 3.69765221e-02] Norma:  10.927576589262472
8  Iteração x= [3.45584277e-03 3.17170310e+01 6.96040198e-02 8.59516377e-01
 3.69616409e-02] Norma:  4.07622454194743
9  Iteração x= [3.43022114e-03 3.13294698e+0

### Método de Broyden

In [106]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F): # n is the number of equations and unknowns
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    Ao = calcJ(J(F), x)
    v = calcF(F, x)
    A = np.linalg.inv(np.array(Ao, dtype=np.float64))
    s = np.dot(-A,v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    while k <= maxIterations:
        w = v
        v = calcF(F, x)
        y = v - w
        z = -np.dot(A,y)
        p = np.dot(-np.transpose(s),z)
        ut = np.dot(np.transpose(s),A)
        A = A + (np.outer((s+z),ut))/p
        s = -np.dot(A,v)
        x = x + s
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(np.array(s, np.float64), 2) < tolerance:
            print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break
        print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        k += 1
        
    return output
             
    

In [108]:
# solve with Broyden's method

x = [10, 10, 10, 10, 10] # para [1, 30, 1, 1, 1] converge na 16!!
Xo = np.transpose(np.array(x)) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 50 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodBroyden(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [ 1 30  1  1  1]
X1 =  [0.00606465637549924 29.9224936866024 0.503653761479361 0.868105595566895
 0.0368326780809132]
2  Iteração x= [0.00508894473219099 29.8733912939840 0.376429180191370 0.859534236017904
 0.0368725659704363] Norma:  109.04634624620046
3  Iteração x= [0.00403355871612505 29.7966483862168 0.216595784546808 0.857642380131586
 0.0369222799403499] Norma:  8.314918711098795
4  Iteração x= [0.00361578324136782 29.7624149031520 0.146038362157734 0.858973009543936
 0.0369439554645721] Norma:  7.028350430491454
5  Iteração x= [0.00340313999509602 29.7531637094191 0.111660869281349 0.859458953344956
 0.0369544577230729] Norma:  1.896447767762025
6  Iteração x= [0.00334704384466043 29.7597331342939 0.102721450283580 0.859477681056703
 0.0369571559860851] Norma:  0.6584095262238857
7  Iteração x= [0.00333479970475373 29.7795042301823 0.100196401975186 0.859455111917542
 0.0369578560229294] Norma:  0.14261518561960956
8  Iteração x= [0.00334727941442896 29.959

### Comparação dos resultados

Ambos os métodos são feitos para inicair de um ponto que já tenha uma aproximação relativamente boa. O método de Newton apresentou uma boa convergência, enquanto o de Broyden não convergiu. De fato, é possível perceber que em outras implementações do método de Broyden, com o ponto inicial dado para as funções dadas, a função não converge.

## ITEM 2

In [110]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
def f1(x):
    return (x[0]*x[1]) + x[0] -(3*x[4])

def f2(x):
    global R
    return 2*x[0]*x[1] +x[0] +3*R[-1]*(x[1]**2) +x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] -R[0]*x[4]

def f3(x):
    global R
    return 2*x[1]*(x[2]**2) +R[-4]*x[1]*x[2] + 2*R[-6]*(x[2]**2) +R[-5]*x[2] -8*x[4]

def f4(x):
    global R
    return R[-2]*x[1]*x[3] +2*(x[3]**2) -4*R[0]*x[4]

def f5(x):
    global R
    return x[0]*x[1] + x[0] +R[-1]*(x[1]**2) + x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] +R[-6]*(x[2]**2) +R[-5]*x[2] +(x[3]**2) -1

Fmatrix = [f1, f2, f3, f4, f5]
    
def F(x):
    return np.transpose(np.array([f1(x),f2(x),f3(x),f4(x),f5(x)], np.float64))


In [111]:
# parcial derivatives by finite differences - FIRST ORDER
def df1(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/h
    
    return df_xk


# parcial derivatives by finite differences - SECOND ORDER
def df2(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x0[k] -= h
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/(2*h)
    
    return df_xk


## define jacobian matrix
def J(Fmatrix, x, order):
    h = .01
    Jmatrix = []
    
    if order == 1:
        for f in Fmatrix:
            Jmatrix.append([df1(f, h, x, 0), df1(f, h, x, 1), df1(f, h, x, 2), df1(f, h, x, 3), df1(f, h, x, 4)])
    elif order == 2:
        for f in Fmatrix:
            Jmatrix.append([df2(f, h, x, 0), df2(f, h, x, 1), df2(f, h, x, 2), df2(f, h, x, 3), df2(f, h, x, 4)])
        
        
    return np.array(Jmatrix, np.float64)    



## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.




### Método de Newton

In [112]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F, Fmatrix):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    print("Começando com Xo = ", x)
    while k <= maxIterations:
        Jx = J(Fmatrix, x, 1)
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(-F(x), np.float64))
        x = x + y
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            k += 1
        
    return output

In [113]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
methodNewton(n, Xo, tolerance, maxIterations, F, Fmatrix)
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [10 10 10 10 10]
1  Iteração x= [0.01065436 9.99921773 5.0035254  5.03889604 0.0364584 ] Norma:  15.768211667204358
2  Iteração x= [ 0.01001145 10.01008704  2.50296252  2.59475134  0.03674464] Norma:  3.496680801986159
3  Iteração x= [9.98392590e-03 1.00835857e+01 1.24778960e+00 1.44170796e+00
 3.68865739e-02] Norma:  1.7059807076471545
4  Iteração x= [9.69190112e-03 1.04286103e+01 6.11188085e-01 9.78727969e-01
 3.69552388e-02] Norma:  0.8594498166837541
5  Iteração x= [8.52579206e-03 1.18233469e+01 2.79490585e-01 8.67730905e-01
 3.69852015e-02] Norma:  1.437927432201653
6  Iteração x= [5.50011558e-03 1.65671346e+01 1.09536521e-01 8.60070632e-01
 3.69914792e-02] Norma:  4.746838284734169
7  Iteração x= [2.98071788e-03 2.72155964e+01 6.22466538e-02 8.59733148e-01
 3.69768143e-02] Norma:  10.648567126855296
8  Iteração x= [3.45110977e-03 3.17482566e+01 6.86179925e-02 8.59520369e-01
 3.69619006e-02] Norma:  4.532664680065862
9  Iteração x= [3.43028257e-03 3.13285458e+0

### Método de Broyden

In [114]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F, Fmatrix): # n is the number of equations and unknowns
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    Ao = J(Fmatrix, x, 2)
    v = F(x)
    A = np.linalg.inv(np.array(Ao, dtype=np.float64))
    s = np.dot(-A,v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -np.dot(A,y)
        p = np.dot(-np.transpose(s),z)
        ut = np.dot(np.transpose(s),A)
        A = A + (np.outer((s+z),ut))/p
        s = -np.dot(A,v)
        x = x + s
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(np.array(s, np.float64), 2) < tolerance:
            print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break
        print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        k += 1
        
    return output
             
    

In [115]:
# solve with Broyden's method

x = [10, 10, 10, 10, 10] # também convergiu em 16 iterações na [1, 30, 1, 1, 1]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
methodBroyden(n, Xo, tolerance, maxIterations, F, Fmatrix)
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [ 1 30  1  1  1]
X1 =  [6.06465638e-03 2.99224937e+01 5.03653761e-01 8.68105596e-01
 3.68326781e-02]
2  Iteração x= [5.08894473e-03 2.98733913e+01 3.76429180e-01 8.59534236e-01
 3.68725660e-02] Norma:  109.04634624621768
3  Iteração x= [4.03355872e-03 2.97966484e+01 2.16595785e-01 8.57642380e-01
 3.69222799e-02] Norma:  8.314918711087211
4  Iteração x= [3.61578324e-03 2.97624149e+01 1.46038362e-01 8.58973010e-01
 3.69439555e-02] Norma:  7.028350430489603
5  Iteração x= [3.40314000e-03 2.97531637e+01 1.11660869e-01 8.59458953e-01
 3.69544577e-02] Norma:  1.8964477677645566
6  Iteração x= [3.34704384e-03 2.97597331e+01 1.02721450e-01 8.59477681e-01
 3.69571560e-02] Norma:  0.658409526226906
7  Iteração x= [3.33479970e-03 2.97795042e+01 1.00196402e-01 8.59455112e-01
 3.69578560e-02] Norma:  0.14261518562132572
8  Iteração x= [3.34727941e-03 2.99593084e+01 9.58423094e-02 8.59427039e-01
 3.69585441e-02] Norma:  0.037534081770690106
9  Iteração x= [3.43333425e-03 3.090746

### Comparação dos resultados

comparar normas

## ITEM 3

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10